In [2]:
# Lab 6 Softmax Classifier
from functools import partial as bind
import numpy as np
import tensorflow as tf
assert(tf.__version__.find('2') == 0)
tf.random.set_seed(777)  # for reproducibility

# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

'''
(101, 16) (101, 1)
'''

nb_classes = 7  # 0 ~ 6

Y_one_hot = tf.keras.utils.to_categorical(y_data, nb_classes)
print("one_hot:", Y_one_hot.shape)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape one_hot:", Y_one_hot.shape)

'''
one_hot: Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape one_hot: Tensor("Reshape:0", shape=(?, 7), dtype=float32)
'''

W = tf.Variable(tf.random.normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random.normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = lambda X: tf.matmul(X, W) + b
hypothesis = lambda X: tf.nn.softmax(logits(X))

prediction = lambda X: tf.argmax(hypothesis(X), 1)
correct_prediction = lambda X: tf.equal(prediction(X), tf.argmax(Y_one_hot, 1))
accuracy = lambda X: tf.reduce_mean(tf.cast(correct_prediction(X), tf.float32))

# Cross entropy cost/loss
@tf.function
def cost(X, Y):
    return tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits(X),
                                                labels=tf.stop_gradient([Y_one_hot]))
    )

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

# Launch graph
for step in range(10001):
    # Minimize
    optimizer.minimize(bind(cost, x_data, y_data), var_list=[W, b])

    if step % 1000 == 0:
        # evaluate training accuracy
        tf.print("Step: ", step, "cost: ", bind(cost, x_data, y_data)(), "accuracy: ", accuracy(x_data))
        
# Let's see if we can predict
pred = prediction(x_data)
# y_data: (N,1) = flatten => (N, ) matches pred.shape
for p, y in zip(pred, y_data.flatten()):
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

'''
Step:     0 Loss: 5.106 Acc: 37.62%
Step:   100 Loss: 0.800 Acc: 79.21%
Step:   200 Loss: 0.486 Acc: 88.12%
...
Step:  1800	Loss: 0.060	Acc: 100.00%
Step:  1900	Loss: 0.057	Acc: 100.00%
Step:  2000	Loss: 0.054	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
...
[True] Prediction: 0 True Y: 0
[True] Prediction: 6 True Y: 6
[True] Prediction: 1 True Y: 1
'''


(101, 16) (101, 1)
one_hot: (101, 7)
reshape one_hot: (101, 7)
Step:  0 cost:  4.33959913 accuracy:  0.158415839
Step:  1000 cost:  0.520363927 accuracy:  0.79207921
Step:  2000 cost:  0.318909109 accuracy:  0.910891116
Step:  3000 cost:  0.235611364 accuracy:  0.940594077
Step:  4000 cost:  0.188909978 accuracy:  0.950495064
Step:  5000 cost:  0.158603594 accuracy:  0.950495064
Step:  6000 cost:  0.13715075 accuracy:  0.990099
Step:  7000 cost:  0.121081986 accuracy:  0.990099
Step:  8000 cost:  0.108556427 accuracy:  1
Step:  9000 cost:  0.0984955281 accuracy:  1
Step:  10000 cost:  0.0902217701 accuracy:  1
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Pre

'\nStep:     0 Loss: 5.106 Acc: 37.62%\nStep:   100 Loss: 0.800 Acc: 79.21%\nStep:   200 Loss: 0.486 Acc: 88.12%\n...\nStep:  1800\tLoss: 0.060\tAcc: 100.00%\nStep:  1900\tLoss: 0.057\tAcc: 100.00%\nStep:  2000\tLoss: 0.054\tAcc: 100.00%\n[True] Prediction: 0 True Y: 0\n[True] Prediction: 0 True Y: 0\n[True] Prediction: 3 True Y: 3\n...\n[True] Prediction: 0 True Y: 0\n[True] Prediction: 6 True Y: 6\n[True] Prediction: 1 True Y: 1\n'